### Samuel Zeleke - CS344 - Homework 3

1. Compute the following values using the restaurant example.

    The **information gain** provided by using the price attribute as the root of the decision tree. Is it more or less 
valuable than the type or patrons attributes computed in class?

    ANSWER:
        Gain(Price) = 1 - Remainder(A)
                    = 1 - ((3/12) * B(1/3) + (2/12) * B(2/2) + (7/12) * B(3/7))
                    = 1 - (0.25 * -((1/3) * lg(1/3) + (2/3) * lg(2/3))
                           + (1/6) * -((2/2) * lg(2/2) + (0/2) * lg(0/2)) # 0*lg(0) is undefined. But, since the right
                                                                          # limit of x*lg(x) as x approaches 0 is 0, 
                                                                          # 0*lg(0) ~ 0
                           + (7/12) * -((3/7) * lg(3/7) + (4/7) * lg(4/7))
                      )
                    = 1 - (0.25 * -(-0.52832 -0.389975) + 
                            (1/6) * -(0) +   
                            (7/12) * -(-0.52388 - 0.46135) 
                          )
                    = 1 - 0.80429
                    = 0.1957
                    
    - Since the gain for `Patrons` is greater than the gain for `Price`. So, asking about `Price` is less valuable than 
    asking about `Patrons`.
    - Since the gain for `Price` is greater than the gain for `Type`, it's more valuable than `Type`.

2. In class, we attempted to create by hand a neural network that computes the XOR function. If this was possible, see 
   if you can simplify the network we built. Consider relaxing the conventions of densely-connected, sequential layers. 
   If it was not possible, give a full explanation why it can’t be done.
   
   ANSWER:
        The XOR function is not linearly separable. (There is no straight line that can segregate [(0, 0), (0, 1), (1, 0), (1, 1)] into [(0, 1), (1, 0)] and [(0, 0), (1, 1)].) So the function can not be represented by a single perceptron or Single-layered network. Instead, we should use two input nodes that learn to label ranges that include (0, 1) and/or (0,1). We should then use an output node that combines the two nodes ranges to only recognize both [(0, 1) and (1, 0)].
        
        Example 
            XOR(a, b) = AND(OR(a, b), NAND(a, b)); OR(a, b) recognizes [(0, 1), (1, 0), and (1, 1)], NAND(a, b) recognizes [(0, 1), (1, 0), and (0, 0)] and AND(...) intersects the two ranges.

3. Use Python/NumPy/Pandas/Keras to load and manipulate the Boston Housing Dataset as follows.

    a. Compute the dimensions of the data structures. Include code to print these values.
    
       ANSWER:

In [110]:
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) =  boston_housing.load_data()

print("training data    dimension: %d, shape: %d x %d" %(train_data.ndim, train_data.shape[0], train_data.shape[1]))
print("training targets dimension: %d, shape: %d" %(train_targets.ndim, train_targets.shape[0]))
print("test data        dimension: %d, shape: %d x %d" %(test_data.ndim, test_data.shape[0], test_data.shape[1]))
print("test targets     dimension: %d, shape: %d" %(test_targets.ndim, test_targets.shape[0]))

training data    dimension: 2, shape: 404 x 13
training targets dimension: 1, shape: 404
test data        dimension: 2, shape: 102 x 13
test targets     dimension: 1, shape: 102


    b. Construct a suitable testing set, training set, and validation set for this data. Submit code to create these 
       datasets but do not include the datasets themselves.
       
       ANSWER:

In [111]:
import pandas
import numpy
import math

# The function partitions input training data into randomized training and validation
# sets where the resulting training dataset is prop_of_training proportion of the original 
# data. 
# Returns: a dictionary containing the numpy arrays for the training, validation, and 
# testing datasets and their targets
#
def partition_data_to_numpy_arrays(
        training, training_targets,
        testing, testing_targets,
        prop_of_training = 0.8):
    
    training_merged = numpy.hstack((training, training_targets.reshape(training_targets.shape[0], 1)))
    numpy.random.shuffle(training_merged)
    
    testing_merged = numpy.hstack((testing, testing_targets.reshape(testing_targets.shape[0], 1)))
    numpy.random.shuffle(testing_merged)

    return {
        "training_data" : training_merged[
                          0:math.floor(prop_of_training * training_merged.shape[0]), 
                          0:training.shape[1] - 1
                          ],\
           "training_targets" : training_merged[
                          0:math.floor(prop_of_training * training_merged.shape[0]), 
                          training.shape[1] - 1
                          ],\
           "validation_data" : training_merged[
                          math.floor((1 - prop_of_training)*training_merged.shape[0]):, 
                          0:training.shape[1] - 1
                          ],\
           "validation_targets" : training_merged[
                          math.floor((1 - prop_of_training)*training_merged.shape[0]):, 
                          training.shape[1] - 1
                          ],\
            "testing_data" : testing_merged[:,0:training.shape[1] - 1],\
            "testing_targets" : testing_merged[:,training.shape[1] - 1]
    }

# The function partitions input training data into randomizedm, labelled training and validation
# sets where the resulting training dataset is prop_of_training proportion of the original 
# data. 
# Returns: a dictionary containing the dataframes for the training, validation, and 
# testing datasets and their targets
#
def partition_data_to_dataframe(
        training, training_targets,
        testing, testing_targets,
        prop_of_training = 0.8):
    
    # labels retrieved from Statlib
    # Variables in order:
    # CRIM     per capita crime rate by town
    # ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
    # INDUS    proportion of non-retail business acres per town
    # CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
    # NOX      nitric oxides concentration (parts per 10 million)
    # RM       average number of rooms per dwelling
    # AGE      proportion of owner-occupied units built prior to 1940
    # DIS      weighted distances to five Boston employment centres
    # RAD      index of accessibility to radial highways
    # TAX      full-value property-tax rate per $10,000
    # PTRATIO  pupil-teacher ratio by town
    # B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
    # LSTAT    % lower status of the population
    # MEDV     Median value of owner-occupied homes in $1000's
    
    labels = [
        "CRIM", "ZN", "INDUS", "CHAS", 
        "NOX", "RM", "AGE", "DIS", 
        "RAD", "TAX", "PTRATIO", "B", 
        "LSTAT","MEDV"
    ]
    
    training_dataframe = pandas.DataFrame()
    testing_dataframe = pandas.DataFrame()
    
    for i in range(len(labels) - 1):
        training_dataframe[labels[i]] = pandas.Series(training[:, i])
        testing_dataframe[labels[i]] = pandas.Series(testing[:, i])
    
    training_dataframe["MEDV"] = pandas.Series(training_targets)
    testing_dataframe["MEDV"] = pandas.Series(testing_targets)
    
    training_dataframe = training_dataframe.reindex(
        numpy.random.permutation(training_dataframe.index)
    )
    
    return {
            "training_data" : training_dataframe.head(
                math.floor(training_dataframe.shape[0] * (prop_of_training)))[labels[:len(labels) - 1]],\
           "training_targets" : pandas.DataFrame(training_dataframe.head(
                math.floor(training_dataframe.shape[0] * (prop_of_training)))[labels[len(labels) - 1]]),\
           "validation_data" : training_dataframe.tail(
                math.floor(training_dataframe.shape[0] * (1 - prop_of_training)))[labels[:len(labels) - 1]],\
           "validation_targets" : pandas.DataFrame(training_dataframe.tail(
                math.floor(training_dataframe.shape[0] * (1 - prop_of_training)))[labels[len(labels) - 1]]),\
            "testing_data" : testing_dataframe[labels[:len(labels) - 1]],\
            "testing_targets" : pandas.DataFrame(testing_dataframe[labels[len(labels) - 1]])
    }

In [112]:
# x = partition_data_to_numpy_arrays(
#     training = train_data, 
#     training_targets = train_targets,
#     testing = test_data, 
#     testing_targets = test_targets)
# y = partition_data_to_dataframe(
#     training = train_data, 
#     training_targets = train_targets,
#     testing = test_data, 
#     testing_targets = test_targets)
# print(x)
# y["training_data"].head()

    c. Create one new synthetic feature that could be useful for machine learning in this domain. Explain what it is and
       why it might be useful, and submit code to add it to the dataset.
       
       ANSWER:
            Synthetic Feature: 
                
                residentiality = ZN : INDUS; 
                
                The feature mesures how "residential" a town is. A higher ratio indicates a town where more land was zoned for residential purposes than for businesses. Houses in quieter, more residential areas (high residentiality ratio) should have a higher value.
                
                However, the two features are fairly negatively correlated. So, the synthetic feature may have an eaggerated influence on the target. 

In [116]:
partitioned_randomized_data = partition_data_to_dataframe(
    training = train_data, 
    training_targets = train_targets,
    testing = test_data, 
    testing_targets = test_targets
)
# Add the synthetic feature the the datasets
for datasets in partitioned_randomized_data:
    # check if the current dataset is a dataframe of examples by using the number of columns
    if partitioned_randomized_data[datasets].shape[1] == 13:
        partitioned_randomized_data[datasets]["RESIDENTIALITY"] = \
            partitioned_randomized_data[datasets]["ZN"] / partitioned_randomized_data[datasets]["INDUS"]
partitioned_randomized_data["training_data"][["ZN", "INDUS", "RESIDENTIALITY"]].head()

,ZN,INDUS,RESIDENTIALITY
369,0.0,18.10,0.000000
61,20.0,6.96,2.873563
384,0.0,4.39,0.000000
277,22.0,5.86,3.754266
113,0.0,9.69,0.000000
